## Set the path of where the csv files are located, as well as initial import statements

In [1]:
csv_data_path = "/home/quinton/Documents/Projects/COMET_Prestudy/scripts/collection/formal_scripts/output_data_mlab_01_2017.csv" # **PLACE CSV FILEPATH HERE**
sep = "\t" # csv delimiter used
small_model = False # use smaller ML model for better performance, less accuracy

In [2]:
from pathlib import Path 
import pandas as pd
import numpy as np

# Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Functions to handle data loading and data writing

In [3]:
def load_data(filename) -> pd.DataFrame:
    filename = Path(filename)
    if not filename.is_file():
        raise FileNotFoundError

    with open(filename.resolve(), 'r', encoding='utf-8') as file:
        data = pd.read_csv(file, sep=sep)
    return data

def load_data_by_year(directory: Path) -> pd.DataFrame:
    for data_file in Path.glob(Path.joinpath(directory, '*.csv')):
            yield load_data(data_file.resolve())

def write_data(filename: Path, data: pd.DataFrame):
    with open(filename.as_uri(), 'w', encoding='utf-8') as f:
        data.to_csv(f, sep=sep)


In [4]:
# obtain the stop words
stopwords = stopwords.words("english")
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [5]:
data = load_data(csv_data_path)['content']
data.head()

0    Hi,  I'm looking for monthly average mobile do...
1    The latest ookla speedtest app (just for apple...
2    We had some unusual results come back today fr...
3    Hi there,  I am working with a group that is i...
4    Hi, It looks like the -server option in ndt7-c...
Name: content, dtype: object

# Content Preprocessing

In [6]:
def lemmatize(posts: pd.DataFrame, allowed_post_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm" if small_model else "en_core_web_trf", disable=["parser", 'ner'])
    texts_out = []

    for post in posts:
        doc = nlp(post)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_post_tags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out

lemmatized_data = lemmatize(data.astype(str))
lemmatized_data[0][:10]

/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Token indices sequence length is longer than the specified maximum sequence length for this model (663 > 512). Running this sequence through the model will result in indexing errors


'look month'

In [7]:
def tokenize_lemma(posts: list):
    final = []
    for post in posts:
        new = gensim.utils.simple_preprocess(post, deacc=True, max_len=512)
        final.append(new)
    return final

tokenized_data = tokenize_lemma(lemmatized_data)
tokenized_data[0][:10]

['look',
 'monthly',
 'average',
 'mobile',
 'download',
 'speed',
 'datum',
 'country',
 'specifcally',
 'onwards']

# Generate id2word dictionary

In [8]:
id2word = corpora.Dictionary(tokenized_data)

corpus = []

for text in tokenized_data:
    new = id2word.doc2bow(text)
    corpus.append(new)

corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

# LDA Visualization

In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=12,
random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated i

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.236581 -0.144073       1        1  24.340197
9     -0.092857  0.191465       2        1  21.203775
5     -0.165547 -0.112091       3        1  12.964782
0     -0.115047  0.145825       4        1  11.927420
3      0.212567 -0.193642       5        1   9.415535
7      0.050172 -0.008181       6        1   7.928474
1      0.021030  0.004744       7        1   4.183686
4     -0.027863 -0.108519       8        1   4.052313
2      0.095890  0.080541       9        1   2.338957
11     0.103002  0.057685      10        1   1.296941
10     0.081011  0.043751      11        1   0.242902
6      0.074222  0.042494      12        1   0.105019, topic_info=                    Term       Freq       Total Category  logprob  loglift
1480     connection_spec  264.00000  264.000000  Default  30.0000  30.0000
1478  client_geolocation  245.00000  245.000000  Default  29.0000  29.0000
149         country_code  206.00000  206.000000  Default  28.0000  28.0000
137                  web  118.00000  118.000000  Default  27.0000  27.0000
47                  test  199.00000  199.000000  Default  26.0000  26.0000
...                  ...        ...         ...      ...      ...      ...
77                client    0.00664   87.867297  Topic12  -7.8989  -2.6316
137                  web    0.00664  118.160261  Topic12  -7.8989  -2.9279
1491                snap    0.00664   87.867136  Topic12  -7.8989  -2.6317
325              already    0.00664    7.990709  Topic12  -7.8989  -0.2341
72                   use    0.00664  164.284738  Topic12  -7.8989  -3.2575

[617 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1990      5  0.809695      a_num
0         1  0.709067       able
0         8  0.253238       able
890       2  0.934119   accepted
242       1  0.886230     access
...     ...       ...        ...
445       1  0.170307      write
445       2  0.145978      write
445       3  0.583910      write
445       6  0.097318      write
2561      4  0.880941  yesterday

[825 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 6, 1, 4, 8, 2, 5, 3, 12, 11, 7])